In [1]:
import pandas as pd
import requests
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
import time
from selenium.webdriver.common.by import By
from PIL import Image
import io
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import json

In [2]:
url = "https://www.pappers.fr/entreprise/{}"

In [26]:
siret = "414817833" #"878990365"

In [27]:
options = Options()
options.set_capability('goog:loggingPrefs', {'performance': 'ALL'})
driver = webdriver.Chrome(options=options,service=ChromeService(ChromeDriverManager().install()))

In [28]:
driver.get(url.format(siret))
time.sleep(5) 
canvas = driver.find_element(By.XPATH, "//canvas[@data-id='layer2-node']")

driver.execute_script("arguments[0].scrollIntoView(true);", canvas)
time.sleep(5) 

logs = driver.get_log("performance")
requests = [json.loads(lr["message"])["message"] for lr in logs]

In [29]:
requests = [req  for req in requests if req['method']=="Network.responseReceived" ]
requests = [req for req in requests if 'cartographie' in req['params']['response']['url']]

In [30]:
request_id = requests[0]["params"]["requestId"]
resp_url = requests[0]["params"]["response"]["url"]
print(f"Caught {resp_url}")
cartographie = driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id})

Caught https://api.pappers.fr/v2/entreprise/cartographie?api_token=97a405f1664a83329a7d89ebf51dc227b90633c4ba4a2575&siren=414817833&inclure_entreprises_dirigees=true&inclure_entreprises_citees=false&inclure_sci=true&autoriser_modifications=true


In [31]:
cartographie

{'base64Encoded': False,
 'body': '{"entreprises":[{"id":"e1","siren":"414817833","nom_entreprise":"GAEC LA FORESTIERE GROUPEMENT AGRICOLE D\'EXPLOITATION EN COMMUN LA FORESTIERE"},{"id":"e2","siren":"322532458","nom_entreprise":"CUMA DE BEAUCHERON"},{"id":"e3","siren":"334912334","nom_entreprise":"CUMA DES BOIS"},{"id":"e4","siren":"330772104","nom_entreprise":"GAEC DE FLIGNE"},{"id":"e5","siren":"483231445","nom_entreprise":"GFA DE FLIGNE"}],"personnes":[{"id":"p1","prenom":"Romain","nom":"Jouteau","niveau":1,"date_naissance":"12/1994"},{"id":"p2","prenom":"Alain","nom":"Jouteau","niveau":1,"date_naissance":"10/1962"},{"id":"p3","prenom":"Dominique","nom":"Percher","niveau":2,"date_naissance":"11/1960"},{"id":"p4","prenom":"Dominique","nom":"Fardeau","niveau":2,"date_naissance":"12/1964"},{"id":"p5","prenom":"Antoine","nom":"Baud","niveau":2,"date_naissance":"03/1996"},{"id":"p6","prenom":"Gerard","nom":"Charruau","niveau":2,"date_naissance":"11/1956"},{"id":"p7","prenom":"Laurent","